# Create Stores Content

In [1]:
import os, sys, random
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import django.db.models as JM
from django.contrib.auth import get_user_model

In [6]:
import emporium.models as EM
import engine.models as GM

import engine.logic.content as CT

In [7]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Get Guild

In [8]:
userMd = get_user_model().objects.filter(user_name='Admin')[0]
guildMd = GM.Guild.objects.GetOrNone(UserFK=userMd, Selected=True)
PD.DataFrame([guildMd.__dict__]).drop(['id', '_state'], axis=1, errors='ignore')

,UserFK_id,Name,LastPlayed,CreateDate,Selected,VaultGold,VaultStone,VaultGems,LastHeist,CampaignWorld,DungeonCheck
0,1,555,2025-01-10,2025-01-08,True,207,71,36,trial,1,2025-01-10


In [12]:
storeLs = GM.MarketStore.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(storeLs).drop(['id', '_state'], axis=1, errors='ignore')

""


In [13]:
# delete magic store for dev

GM.MarketStore.objects.all().delete()

(9, {'engine.MarketStore': 9})

In [ ]:
storeLs = GM.ThiefUnlocked.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(storeLs).drop(['id', '_state'], axis=1, errors='ignore')

In [ ]:
# unlocked blueprints report

unlockThief = EM.UnlockableThief.objects.filter(Stars__gt=1).values()

PD.DataFrame(unlockThief)#.drop(['id', '_state'], axis=1, errors='ignore')

In [ ]:
for th in unlockThief:
    th['IconCode'] = f"class-{th['Class'].lower()}-s{th['Stars']}"
    checkUnlock = GM.ThiefUnlocked.objects.GetOrNone(GuildFK=guildMd, ThiefFK__ResourceId=th['ResourceId'])
    th['Unlocked'] = True if checkUnlock else False
    
PD.DataFrame(unlockThief)